## Import Library

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
project_path = "/content/drive/MyDrive/ComBio"
root_folder = f"{project_path}/Dataset"
output_folder = f"{project_path}/Dataset/Processed"

Mounted at /content/drive


## Merge Data

In [ ]:
def clean_nhanes_codes(df):
    df_cleaned = df.copy()
    for col in df_cleaned.columns:
        col_data = df_cleaned[col]
        is_categorical = False
        if col.endswith(('LC', 'FC', 'SI', 'IND', 'CODE')):
            is_categorical = True

        elif col_data.nunique() < 10 and pd.api.types.is_numeric_dtype(col_data):
            is_categorical = True

        if pd.api.types.is_numeric_dtype(col_data) and not is_categorical:
            extreme_mask = (col_data.abs() < 1e-10) & (col_data != 0)
            if extreme_mask.any():
                print(f"Kolom {col}: mengganti {extreme_mask.sum()} nilai ekstrim dengan NaN")
                df_cleaned.loc[extreme_mask, col] = np.nan

        if pd.api.types.is_numeric_dtype(col_data):
            # Nilai negatif dalam NHANES:
            # -1: Refused
            # -2: Don't know
            # -7: Refused to answer
            # -9: Don't know/missing
            nhanes_codes = [-1, -2, -7, -9]
            for code in nhanes_codes:
                code_mask = col_data == code
                if code_mask.any():
                    print(f"Kolom {col}: mengganti {code_mask.sum()} nilai kode {code} dengan NaN")
                    df_cleaned.loc[code_mask, col] = np.nan

    return df_cleaned

def process_nhanes_period(folder, suffix, filename_csv):
    print(f"\n{'='*50}")
    print(f"MEMPROSES DATA NHANES DI FOLDER: {folder}")
    print(f"{'='*50}")

    try:
        if not os.path.exists(folder):
            print(f"Error: Folder {folder} tidak ditemukan!")
            return False

        datasets = {}
        files_to_read = {
            "demo": f"DEMO{suffix}.XPT",
            "biopro": f"BIOPRO{suffix}.XPT",
            "hepc": f"HEPC{suffix}.XPT",
            "mcq": f"MCQ{suffix}.XPT",
            "alq": f"ALQ{suffix}.XPT",
            "duq": f"DUQ{suffix}.XPT",
        }

        for name, filename in files_to_read.items():
            file_path = os.path.join(folder, filename)
            if not os.path.exists(file_path):
                print(f"Warning: File {filename} tidak ditemukan di {folder}")
                print(f"Coba mencari file alternatif...")

                possible_files = [f for f in os.listdir(folder) if f.lower().startswith(name.lower())]
                if possible_files:
                    alt_file = possible_files[0]
                    print(f"Menggunakan file alternatif: {alt_file}")
                    file_path = os.path.join(folder, alt_file)
                else:
                    print(f"Tidak ditemukan file alternatif untuk {name}. Melewati dataset ini.")
                    continue

            try:
                print(f"Membaca file {file_path}...")
                datasets[name] = pd.read_sas(file_path)
                print(f"Berhasil membaca {filename} dengan {len(datasets[name])} baris dan {len(datasets[name].columns)} kolom")
            except Exception as e:
                print(f"Error saat membaca {filename}: {str(e)}")
                print("Melewati dataset ini.")
                continue

        if len(datasets) < 2:
            print("Terlalu sedikit dataset yang berhasil dibaca untuk melakukan merge.")
            print("Minimal diperlukan 2 dataset untuk melanjutkan.")
            return False

        print("\nMembersihkan nilai kode khusus di setiap dataset...")
        cleaned_datasets = {}
        for name, data in datasets.items():
            cleaned_datasets[name] = clean_nhanes_codes(data)

        print("\nMelakukan merge dataset...")
        base_dataset_name = "demo" if "demo" in cleaned_datasets else list(cleaned_datasets.keys())[0]
        merged_df = cleaned_datasets[base_dataset_name]
        print(f"Dataset awal ({base_dataset_name}): {merged_df.shape}")

        for name, data in cleaned_datasets.items():
            if name == base_dataset_name:
                continue

            prev_shape = merged_df.shape
            how_join = "inner" if name in ["biopro", "hepc"] else "left"
            merged_df = merged_df.merge(data, on="SEQN", how=how_join)
            print(f"Setelah merge dengan {name}: {merged_df.shape} (kehilangan {prev_shape[0] - merged_df.shape[0]} baris)")

        print(f"\nDataset gabungan final shape: {merged_df.shape}")

        print("Memeriksa nilai khusus setelah merge...")
        df_final = clean_nhanes_codes(merged_df)

        os.makedirs(output_folder, exist_ok=True)

        output_filename = f"{filename_csv}.csv"
        output_path = os.path.join(output_folder, output_filename)
        print(f"Menyimpan hasil ke {output_path}...")
        df_final.to_csv(output_path, index=False)
        print(f"Dataset berhasil disimpan!")

        print("\nInformasi tentang dataset:")
        for dtype, count in df_final.dtypes.value_counts().items():
            print(f"- {count} kolom dengan tipe data {dtype}")

        return True

    except Exception as e:
        print(f"Error saat memproses data di folder {folder}: {str(e)}")
        return False

In [ ]:
print("PEMROSESAN DATA NHANES UNTUK PERIODE 2005-2006 SAMPAI 2017-2018")
print("===============================================================")


os.makedirs(output_folder, exist_ok=True)

periods = [
    (f"{root_folder}/2005_2006", "_D", "2005-2006"),  # Suffix D
    (f"{root_folder}/2007_2008", "_E", "2007-2008"),  # Suffix E
    (f"{root_folder}/2009_2010", "_F", "2009-2010"),  # Suffix F
    (f"{root_folder}/2011_2012", "_G", "2011-2012"),  # Suffix G
    (f"{root_folder}/2013_2014", "_H", "2013-2014"),  # Suffix H
    (f"{root_folder}/2015_2016", "_I", "2015-2016"),  # Suffix I
    (f"{root_folder}/2017_2018", "_J", "2017-2018"),  # Suffix J
]

success_count = 0
total_periods = len(periods)

for folder, suffix, filename in periods:
    print(f"\nMemproses periode {filename} dengan suffix {suffix}...")
    if process_nhanes_period(folder, suffix, filename):
        success_count += 1
        print(f"\n✅ Data {filename} berhasil diproses")
    else:
        print(f"\n❌ Gagal memproses data {filename}")

print(f"\nPROSES SELESAI. Berhasil memproses {success_count} dari {total_periods} periode.")
